# Load Package

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

import os
cur_path = "/content/drive/MyDrive/Bigdataproject/"
os.chdir(cur_path)
!pwd

/content/drive/.shortcut-targets-by-id/17vQsBka11gF9J6q62UIKd778-o_uQJUU/Bigdataproject


In [ ]:
!pip install pyspark
!pip install pyspark[sql]
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 17.6 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=5df4c3e646ad7352742c520a6bef5b45547b8405ce0fad0f7880222cb0ac7248
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns

In [ ]:
import pyspark as spark
import time
from operator import add
from pyspark.sql import SparkSession

In [ ]:
import numpy as np
import random

# Step 0: Read Data

## Set spark session

In [ ]:
# set spark session
ss = (SparkSession
  .builder
  .master("local[5]")
  .appName("fqproject")
  .getOrCreate())


## Set spark context

In [ ]:
# set spark session
sc = (spark
  .SparkContext
  .getOrCreate(spark
    .SparkConf()
    .setAppName("fqproject")
    .setMaster('local[*]')
    .set('spark.executor.memory', '4G')
    .set('spark.driver.memory', '4G')
    .set('spark.driver.maxResultSize', '4G')))

## Read reads.fq

In [ ]:
# read reads.fq and extract the second line of every four lines.
reads = (sc
  .textFile(cur_path + "data/reads.fq")
  .zipWithIndex()
  .filter(lambda x: (x[1]+1)%2==0 and (x[1]+1)%4!=0)
  .map(lambda x: x[0]))

In [ ]:
reads.take(10)

['TCCTTACTGGTTTTGCAGGTAACTTATAGAGTATTTCCACTTCCCTTCTCCTATCCCTTGAAAAATTGTCATTTATTTCTCTTATCCATATGGCATAATC',
 'GGTTTTTCAGGTAACTTATAGAGTATTTCCACTTCCCTTCTCCTATCCCTGGAAAAATTGTCATTTATTTCTCTTATCCATATGGCATAATCAAAGAATA',
 'AGGTAACTTATAGAGTACTTCCACTTCCCTTCTCCTATCCCTTGAAAAATTGTCATTGATTTCTCTTATCCATATGGCATAATCAAAGAATAAATTGGTG',
 'CACTTCCCTTCTACTATCCCTTGAAAAATTGTCATTTATTTCTCTTATCCATATGGCATAATCAAAGAATAAATTGGTGATATTTGTTCAAAAATCCATG',
 'CCTTCTCCTATCCCTTGAAAAATTGTCATTTATTTCCCTTATCCATATGGCATAATCAAAGAATAAATTGGTGATATTTGTTCAAAAATCCATGCCTTTT',
 'ATCCCTTGAAAAATTGTCATTTATTTCTCTTATCCATATGGCATAATCAAAGAATAAATTGGTGATATTTGTTCAAAAATCCGTGCCTATTAGATTCATT',
 'TCTTATCCATATGGCATAATCAAAGAATAAATTGGTGATATTTGTTCAAAAATCCATGCCTATTAGATTCATTTAGAATATAAAAAAATTTTATTTTATT',
 'TATCCATATGGCATAATCAAAGAATCAATTGTTGATATTTGTTCAAAAATCCATGCCTATTAGATTCATTTAGAATATAAAAAAATTTTATTTTATTTTC',
 'TCAAAGAATAAATTGGTGATATTTGTTCAAAAATCCATGCCTATTAGATTCATTTAGATTATAAAAAAATTTTATTTTATTTTCACTTATTTCTTCTCCA',
 'TGAATAAATTGGTGATATTTGTTCAAAAATCCATGCCTACTAGATTCATTTAG

## Read reference.fa

In [ ]:
# read reference
reference = (ss
  .read
  .csv(
      cur_path + "data/reference_chr21_20000000_20050000.fa",
      inferSchema=True,
      header=True))

In [ ]:
reference.show(10)

+--------+--------+--------------------+
|   start|     end|            sequence|
+--------+--------+--------------------+
|20000000|20000100|CCCTTCTCCTATCCCTT...|
|20000100|20000200|TAGATTCATTTAGAATA...|
|20000200|20000300|TTCTTCTTCCTGAAGAA...|
|20000300|20000400|ACTTTTCAAGGATAGTT...|
|20000400|20000500|CTGACAGGACTTCTGCC...|
|20000500|20000600|TTTCCTTTTTTTTTTCT...|
|20000600|20000700|ATTATAAAAAGGGAGGG...|
|20000700|20000800|TTCTTTTCTTTTTCTCC...|
|20000800|20000900|ATAAATTTCTGCTTGAA...|
|20000900|20001000|TTCGTTAGTGTTTTTTA...|
+--------+--------+--------------------+
only showing top 10 rows



In [ ]:
reference.count()

500

# Step 1: Cut reads into kmers = 15

## Define K-mers

In [ ]:
# input a string s and cut k-mers
def extract_kmers(s: str, k: int = 15) -> list:
    return [s[i:i+k] for i in range(len(s)-k+1)]

## Map K-mers to reads

In [ ]:
# K-mers of reads
reads_kmers = (reads
  .map(extract_kmers))

In [ ]:
reads_kmers.take(2)

[['TCCTTACTGGTTTTG',
  'CCTTACTGGTTTTGC',
  'CTTACTGGTTTTGCA',
  'TTACTGGTTTTGCAG',
  'TACTGGTTTTGCAGG',
  'ACTGGTTTTGCAGGT',
  'CTGGTTTTGCAGGTA',
  'TGGTTTTGCAGGTAA',
  'GGTTTTGCAGGTAAC',
  'GTTTTGCAGGTAACT',
  'TTTTGCAGGTAACTT',
  'TTTGCAGGTAACTTA',
  'TTGCAGGTAACTTAT',
  'TGCAGGTAACTTATA',
  'GCAGGTAACTTATAG',
  'CAGGTAACTTATAGA',
  'AGGTAACTTATAGAG',
  'GGTAACTTATAGAGT',
  'GTAACTTATAGAGTA',
  'TAACTTATAGAGTAT',
  'AACTTATAGAGTATT',
  'ACTTATAGAGTATTT',
  'CTTATAGAGTATTTC',
  'TTATAGAGTATTTCC',
  'TATAGAGTATTTCCA',
  'ATAGAGTATTTCCAC',
  'TAGAGTATTTCCACT',
  'AGAGTATTTCCACTT',
  'GAGTATTTCCACTTC',
  'AGTATTTCCACTTCC',
  'GTATTTCCACTTCCC',
  'TATTTCCACTTCCCT',
  'ATTTCCACTTCCCTT',
  'TTTCCACTTCCCTTC',
  'TTCCACTTCCCTTCT',
  'TCCACTTCCCTTCTC',
  'CCACTTCCCTTCTCC',
  'CACTTCCCTTCTCCT',
  'ACTTCCCTTCTCCTA',
  'CTTCCCTTCTCCTAT',
  'TTCCCTTCTCCTATC',
  'TCCCTTCTCCTATCC',
  'CCCTTCTCCTATCCC',
  'CCTTCTCCTATCCCT',
  'CTTCTCCTATCCCTT',
  'TTCTCCTATCCCTTG',
  'TCTCCTATCCCTTGA',
  'CTCCTATCCC

# Step 2: Cut reference bins into kmers = 15

In [ ]:
# K-mers of reference
reference_kmers = (reference
  .select('sequence')
  .rdd
  .flatMap(lambda x: x)
  .map(extract_kmers))

In [ ]:
reference_kmers.take(2)

[['CCCTTCTCCTATCCC',
  'CCTTCTCCTATCCCT',
  'CTTCTCCTATCCCTT',
  'TTCTCCTATCCCTTG',
  'TCTCCTATCCCTTGA',
  'CTCCTATCCCTTGAA',
  'TCCTATCCCTTGAAA',
  'CCTATCCCTTGAAAA',
  'CTATCCCTTGAAAAA',
  'TATCCCTTGAAAAAT',
  'ATCCCTTGAAAAATT',
  'TCCCTTGAAAAATTG',
  'CCCTTGAAAAATTGT',
  'CCTTGAAAAATTGTC',
  'CTTGAAAAATTGTCA',
  'TTGAAAAATTGTCAT',
  'TGAAAAATTGTCATT',
  'GAAAAATTGTCATTT',
  'AAAAATTGTCATTTA',
  'AAAATTGTCATTTAT',
  'AAATTGTCATTTATT',
  'AATTGTCATTTATTT',
  'ATTGTCATTTATTTC',
  'TTGTCATTTATTTCT',
  'TGTCATTTATTTCTC',
  'GTCATTTATTTCTCT',
  'TCATTTATTTCTCTT',
  'CATTTATTTCTCTTA',
  'ATTTATTTCTCTTAT',
  'TTTATTTCTCTTATC',
  'TTATTTCTCTTATCC',
  'TATTTCTCTTATCCA',
  'ATTTCTCTTATCCAT',
  'TTTCTCTTATCCATA',
  'TTCTCTTATCCATAT',
  'TCTCTTATCCATATG',
  'CTCTTATCCATATGG',
  'TCTTATCCATATGGC',
  'CTTATCCATATGGCA',
  'TTATCCATATGGCAT',
  'TATCCATATGGCATA',
  'ATCCATATGGCATAA',
  'TCCATATGGCATAAT',
  'CCATATGGCATAATC',
  'CATATGGCATAATCA',
  'ATATGGCATAATCAA',
  'TATGGCATAATCAAA',
  'ATGGCATAAT

# Step 3: Collect all kmers and build a distinct kmer set

• Hint: you can use python “set” function

• Report the number of distinct kmers (N)

In [ ]:
# Merge and distinct, collect to a set
kmers = set(reads_kmers 
  .union(reference_kmers)
  .flatMap(lambda x: x) 
  .distinct()
  .collect())

In [ ]:
len(kmers)

72530

**The number of distinct kmers is 72,530**


In [ ]:
reads

PythonRDD[40] at RDD at PythonRDD.scala:53

# Step4

In [ ]:
from pyspark.ml.linalg import Vectors
import pandas as pd

In [ ]:

# turn reads to list 
r_list = reads.collect()

In [ ]:
# check feature vector of every kmer in each read
def check_kmer_features(kmer):
  m = []
  

  for i in range(len(r_list)):
    if kmer in r_list[i]:
      m.append(i)
  v = [1.0 for i in range(len(m))]
  return (kmer,Vectors.dense(Vectors.sparse(len(r_list),m,v)))

In [ ]:
kmers_rdd = sc.parallelize(list(kmers))

In [ ]:
reads_features = kmers_rdd.map(lambda x: check_kmer_features(x))

In [ ]:
reads_features_df = reads_features.toDF(['kmer','read_features'])

In [ ]:
#generate the dataframe with sparse vector
reads_features_df.show(1)

+---------------+--------------------+
|           kmer|       read_features|
+---------------+--------------------+
|CCTGTTGAAGGCCAT|[0.0,0.0,0.0,0.0,...|
+---------------+--------------------+
only showing top 1 row



In [ ]:
from pyspark.sql.functions import col
from pyspark.ml.functions import vector_to_array

In [ ]:
reads_features_df = reads_features_df.withColumn("read", vector_to_array("read_features")).select(["kmer"] + [col("read")[i] for i in range(2000)])

In [ ]:
read_name = {r_list[i]: 'reads' + str(i) for i in range(len(r_list))}

In [ ]:
# old code
def check_kmer_features_old1(read):
  m = []
  kmers_list = list(kmers)
  for i in range(len(kmers_list)):
    if kmers_list[i] in read:
      m.append(i)
  v = [1.0 for i in range(len(m))]
  return (read_name[read],Vectors.sparse(len(kmers_list),m,v))


In [ ]:
reads_features = reads.map(lambda x: check_kmer_features_old1(x))

In [ ]:
reads_features.take(2)

[('reads0',
  SparseVector(72530, {172: 1.0, 851: 1.0, 1696: 1.0, 2460: 1.0, 4002: 1.0, 4611: 1.0, 4892: 1.0, 6085: 1.0, 6582: 1.0, 7638: 1.0, 7930: 1.0, 9003: 1.0, 11635: 1.0, 13193: 1.0, 16383: 1.0, 17643: 1.0, 18179: 1.0, 18278: 1.0, 19246: 1.0, 19553: 1.0, 20322: 1.0, 20400: 1.0, 21792: 1.0, 22091: 1.0, 23404: 1.0, 23745: 1.0, 23949: 1.0, 23970: 1.0, 24726: 1.0, 25534: 1.0, 25949: 1.0, 26535: 1.0, 26776: 1.0, 27996: 1.0, 28211: 1.0, 30111: 1.0, 31479: 1.0, 34065: 1.0, 34677: 1.0, 35105: 1.0, 36098: 1.0, 37866: 1.0, 38470: 1.0, 40054: 1.0, 40483: 1.0, 40773: 1.0, 41127: 1.0, 41696: 1.0, 42503: 1.0, 42575: 1.0, 42917: 1.0, 43070: 1.0, 43495: 1.0, 44224: 1.0, 45103: 1.0, 46246: 1.0, 50538: 1.0, 51935: 1.0, 53797: 1.0, 54047: 1.0, 54197: 1.0, 54917: 1.0, 55895: 1.0, 56222: 1.0, 58227: 1.0, 59428: 1.0, 59622: 1.0, 61052: 1.0, 61058: 1.0, 61213: 1.0, 61337: 1.0, 61470: 1.0, 61885: 1.0, 61974: 1.0, 62578: 1.0, 63348: 1.0, 63440: 1.0, 64999: 1.0, 65353: 1.0, 65724: 1.0, 66310: 1.0, 67208: 

# Step5

In [ ]:
reference_rdd = reference.select("sequence").rdd.map(lambda x: x['sequence'])

In [ ]:
reference_list = reference_rdd.collect()
reference_list[0]

'CCCTTCTCCTATCCCTTGAAAAATTGTCATTTATTTCTCTTATCCATATGGCATAATCAAAGAATAAATTGGTGATATTTGTTCAAAAATCCATGCCTAT'

In [ ]:
#generate the name of columns
start_rdd = reference.select("start").rdd.map(lambda x: x['start'])
end_rdd = reference.select("end").rdd.map(lambda x: x['end'])
s_list = start_rdd.collect()
e_list = end_rdd.collect()
reference_name = ['bin' + str(s_list[i]) + '_' +  str(e_list[i]) for i in range(len(s_list))]
reference_name[0]

'bin20000000_20000100'

In [ ]:
# check feature vector of every kmer in each reference
def check_kmer_read_features_sequence(kmer):
  m = []
  for i in range(len(reference_list)):
    if kmer in reference_list[i]:
      m.append(i)
  v = [1.0 for i in range(len(m))]
  return (kmer,Vectors.dense(Vectors.sparse(len(reference_list),m,v)))

In [ ]:
reference_features = kmers_rdd.map(lambda x:check_kmer_read_features_sequence(x))

In [ ]:
# turn to dataframe
reference_features_df = reference_features.toDF(['kmer','reference_features'])

In [ ]:
reference_features_df.show(1) 

+---------------+--------------------+
|           kmer|  reference_features|
+---------------+--------------------+
|CCTGTTGAAGGCCAT|[0.0,0.0,0.0,0.0,...|
+---------------+--------------------+
only showing top 1 row



In [ ]:
# expand the dense vector to different collomns
reference_features_df = reference_features_df.withColumn("xs", vector_to_array("reference_features")).select(["kmer"] + [col("xs")[i] for i in range(len(reference_name))])

In [ ]:
reference_name = ['kmer'] + reference_name
reference_features_df = reference_features_df.select([col(c).alias(reference_name[i]) for i, c in enumerate(reference_features_df.columns)])

In [ ]:
reference_name_d = {reference_list[i]: reference_name[i+1] for i in range(len(reference_list))}
len(reference_name_d)

500

In [ ]:
# old code
def check_kmer_features_old(read):
  m = []
  kmers_list = list(kmers)
  for i in range(len(kmers_list)):
    if kmers_list[i] in read:
      m.append(i)
  v = [1.0 for i in range(len(m))]
  return (reference_name_d[read],Vectors.sparse(len(kmers_list),m,v))

In [ ]:
# old code 
reference_features = reference_rdd.map(lambda x: check_kmer_features_old(x))
reference_features.take(1)

[('bin20000000_20000100',
  SparseVector(72530, {172: 1.0, 851: 1.0, 2589: 1.0, 4996: 1.0, 6085: 1.0, 7345: 1.0, 7638: 1.0, 8856: 1.0, 9003: 1.0, 9265: 1.0, 11635: 1.0, 13476: 1.0, 13825: 1.0, 14091: 1.0, 15250: 1.0, 15325: 1.0, 15621: 1.0, 15832: 1.0, 16310: 1.0, 16660: 1.0, 17214: 1.0, 17643: 1.0, 18179: 1.0, 19246: 1.0, 19553: 1.0, 20322: 1.0, 20400: 1.0, 21792: 1.0, 22091: 1.0, 23017: 1.0, 25534: 1.0, 26030: 1.0, 26535: 1.0, 27996: 1.0, 33468: 1.0, 33776: 1.0, 34065: 1.0, 34665: 1.0, 36137: 1.0, 38470: 1.0, 38736: 1.0, 40483: 1.0, 40773: 1.0, 41127: 1.0, 41696: 1.0, 42055: 1.0, 42503: 1.0, 42575: 1.0, 43070: 1.0, 43799: 1.0, 44224: 1.0, 45103: 1.0, 46246: 1.0, 46882: 1.0, 47325: 1.0, 48122: 1.0, 49416: 1.0, 49880: 1.0, 50174: 1.0, 51935: 1.0, 53098: 1.0, 53764: 1.0, 53797: 1.0, 54047: 1.0, 54097: 1.0, 57833: 1.0, 58227: 1.0, 58702: 1.0, 59428: 1.0, 61052: 1.0, 61213: 1.0, 61974: 1.0, 62465: 1.0, 62578: 1.0, 63348: 1.0, 63440: 1.0, 65030: 1.0, 65353: 1.0, 66310: 1.0, 67536: 1.0, 675


# Step 6: Use minhash to reduce the feature dimension for reads and reference bins

In [ ]:
# define minhash
def minhash(hashtable: list, entry: tuple) -> tuple:
  ans = []
  for ls in hashtable:
    for i in ls:
      if entry[1][i]==1:
        ans.append(i)
        break
  return (entry[0], ans)

In [ ]:
# create hash table
def minhash_table(n: int) -> list:
  ans = []
  for i in range(n):
    ans.append(random.sample(range(72530), 72530))
  return ans

In [ ]:
# set seed
random.seed(114514)

In [ ]:
# create hash table
hashtable = minhash_table(1000)

In [ ]:
# run minhash on reads
start = time.time()
print(time.localtime(start))
read_sig_temp = reads_features.map(lambda x: minhash(hashtable, x))
print(time.time()-start)

time.struct_time(tm_year=2023, tm_mon=4, tm_mday=10, tm_hour=19, tm_min=48, tm_sec=19, tm_wday=0, tm_yday=100, tm_isdst=0)
0.0011811256408691406


In [ ]:
# run minhash on references
start = time.time()
print(time.localtime(start))
ref_sig_temp = reference_features.map(lambda x: minhash(hashtable, x))
print(time.time()-start)

time.struct_time(tm_year=2023, tm_mon=4, tm_mday=10, tm_hour=19, tm_min=51, tm_sec=4, tm_wday=0, tm_yday=100, tm_isdst=0)
0.0007219314575195312


In [ ]:
len(read_sig_temp.take(1)[0][1])

1000

In [ ]:
len(ref_sig_temp.take(1)[0][1])

1000